In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import random
import time 
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## 先只爬一篇文章的資訊、推文留言

In [184]:
payload={
'from':'/bbs/'+ 'Gossiping' +'/index.html',
'yes':'yes' 
}

url = 'https://www.ptt.cc/bbs/Examination/M.1582260742.A.8D9.html'
#url = 'https://www.ptt.cc/bbs/sex/M.1582985884.A.88A.html'
#url = 'https://www.ptt.cc/bbs/LoL/M.1583142390.A.3A0.html'
#破除18禁的限制
rs = requests.session()
req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
req = rs.get('https://www.ptt.cc/bbs/'+ 'sex' +'/index.html',verify = False)
req = rs.get(url, verify=False)

In [4]:
#200 means get success, 404 means fail
req.status_code

200

In [5]:
web_content = req.text
#以美味湯解析html
soup = BeautifulSoup(web_content, 'lxml')

In [6]:
main_content = soup.find('div',id="main-content")
metas = main_content.select('div.article-metaline')

In [7]:
author = metas[0].select('span.article-meta-value')[0].text
title = metas[1].select('span.article-meta-value')[0].text
post_date = metas[2].select('span.article-meta-value')[0].text

In [8]:
author_ID = author.split('(',1)[0].replace(' ','')
author_name = author.split('(',1)[1].replace(')','')

In [9]:
#content篩選出文章內文
content = soup.find(id="main-content").text
target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
content = content.split(target_content)
date = soup.select('.article-meta-value')[3].text
content = content[0].split(date)
content = content[1].replace('\n', '  ').replace('\t', '  ')

In [10]:
note = soup.select('span.f2')

In [11]:
org_url = note[1].text.replace('※ 文章網址: ','')

In [12]:
createdTime = dt.datetime.today().strftime("%d/%m/%Y %H:%M:%S")

In [13]:
createdTime

'03/03/2020 19:07:05'

In [14]:
post_info = []
post_info.append({'authorId':author_ID, 'authorName':author_name, 'title':title, 'publishedTime':post_date
                  , 'content':content, 'canonicalUrl':org_url, 'createdTime':createdTime})

In [15]:
post_info = pd.DataFrame(post_info)

In [16]:
pushes = main_content.find_all('div', class_='push')

In [19]:
messages = []
for push in pushes:
    push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
    push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
    # if find is None: find().strings -> list -> ' '.join; else the current way
    push_content = push.find('span', 'push-content').strings
    push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
    push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')    
    messages.append( {'canonicalUrl':org_url, 'push_tag': push_tag, 'commentId': push_userid
                      , 'commentContent': push_content, 'commentTime': push_ipdatetime} )


In [20]:
messages = pd.DataFrame(messages)

In [21]:
messages

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/Examination/M.158226074...,推,milk7054,第一年剛開始打底就別想太多，先當練筆,02/21 13:04
1,https://www.ptt.cc/bbs/Examination/M.158226074...,→,milk7054,6科專業，每科20章，一天看一科，這樣根本來不及,02/21 13:09
2,https://www.ptt.cc/bbs/Examination/M.158226074...,→,milk7054,光函授看完第一輪就差不多要考試,02/21 13:10
3,https://www.ptt.cc/bbs/Examination/M.158226074...,推,needeng,準時下班，怎會沒辦法看書？平日不行的話，你放假有看書,02/21 13:42
4,https://www.ptt.cc/bbs/Examination/M.158226074...,→,needeng,？你的企圖心不夠。,02/21 13:42
...,...,...,...,...,...
65,https://www.ptt.cc/bbs/Examination/M.158226074...,推,Curry16,人家全職考生窮到每天被家人念，你有薪水領，本來就要比,02/24 07:26
66,https://www.ptt.cc/bbs/Examination/M.158226074...,→,Curry16,較努力,02/24 07:26
67,https://www.ptt.cc/bbs/Examination/M.158226074...,推,h14753951,"過太爽就會沒動力念書,沒有壓力就沒有動力",02/24 12:23
68,https://www.ptt.cc/bbs/Examination/M.158226074...,推,h14753951,你應該好好找一下動力來源,02/24 12:27


In [22]:
pd.merge(post_info, messages, how='left', on='canonicalUrl')

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime,push_tag,commentId,commentContent,commentTime
0,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,milk7054,第一年剛開始打底就別想太多，先當練筆,02/21 13:04
1,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,milk7054,6科專業，每科20章，一天看一科，這樣根本來不及,02/21 13:09
2,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,milk7054,光函授看完第一輪就差不多要考試,02/21 13:10
3,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,needeng,準時下班，怎會沒辦法看書？平日不行的話，你放假有看書,02/21 13:42
4,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,needeng,？你的企圖心不夠。,02/21 13:42
...,...,...,...,...,...,...,...,...,...,...,...
65,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,Curry16,人家全職考生窮到每天被家人念，你有薪水領，本來就要比,02/24 07:26
66,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,→,Curry16,較努力,02/24 07:26
67,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,h14753951,"過太爽就會沒動力念書,沒有壓力就沒有動力",02/24 12:23
68,oiuyman,man,[請益] 兼職考該怎麼安排時間,Fri Feb 21 12:52:20 2020,以下代po 已經考上初等考 想在準備高普考 雖然工作算準時下班 ...,https://www.ptt.cc/bbs/Examination/M.158226074...,03/03/2020 19:07:05,推,h14753951,你應該好好找一下動力來源,02/24 12:27


## 抓某個看板的某個頁面的所有文章網址

In [28]:
payload={
'from':'/bbs/'+ 'sex' +'/index.html',
'yes':'yes' 
}

url = 'https://www.ptt.cc/bbs/Examination/index.html'
#破除18禁的限制
rs = requests.session()
req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
req = rs.get('https://www.ptt.cc/bbs/'+ 'sex' +'/index.html',verify = False)
req = rs.get(url, verify=False)

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [29]:
#200 means get success, 404 means fail
req.status_code

200

In [39]:
web_content = req.text
#以美味湯解析html
soup = BeautifulSoup(web_content, 'lxml')

In [48]:
post_list = soup.find_all('div',class_="title")

In [52]:
url_list = []
for item in post_list:
    try:
        url_list.append('https://www.ptt.cc'+item.select_one("a").get("href"))
    except: #刪文抓不到href
        pass

In [54]:
#得到該頁面的所有url
url_list

['https://www.ptt.cc/bbs/Examination/M.1582969553.A.FEE.html',
 'https://www.ptt.cc/bbs/Examination/M.1582979626.A.822.html',
 'https://www.ptt.cc/bbs/Examination/M.1583037348.A.AC9.html',
 'https://www.ptt.cc/bbs/Examination/M.1583038395.A.B8B.html',
 'https://www.ptt.cc/bbs/Examination/M.1583050146.A.269.html',
 'https://www.ptt.cc/bbs/Examination/M.1583050709.A.EB4.html',
 'https://www.ptt.cc/bbs/Examination/M.1583057947.A.E9E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583069018.A.29E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583075191.A.D9E.html',
 'https://www.ptt.cc/bbs/Examination/M.1583120282.A.3DB.html',
 'https://www.ptt.cc/bbs/Examination/M.1583133271.A.B3B.html',
 'https://www.ptt.cc/bbs/Examination/M.1583149220.A.F93.html',
 'https://www.ptt.cc/bbs/Examination/M.1583151717.A.0B7.html',
 'https://www.ptt.cc/bbs/Examination/M.1583160130.A.221.html',
 'https://www.ptt.cc/bbs/Examination/M.1583201813.A.888.html',
 'https://www.ptt.cc/bbs/Examination/M.1583216252.A.6F5

* 抓取該看板的總頁數

In [79]:
board = 'sex'
pre_num = soup.select('div.btn-group > a')[3]['href'].replace('/bbs/'+board+'/index','') ##
pre_num = pre_num.replace('.html','')
total_page_index = pd.to_numeric(pre_num)+1

ValueError: Unable to parse string "/bbs/Examination/index3007" at position 0

In [56]:
total_page_index

3008

In [57]:
for i in range(3008,3002,-1):
    print(i)

3008
3007
3006
3005
3004
3003


# 得到一篇文章資訊的function

In [2]:
def get_post_data(post_url):
    payload={
    'from':'/bbs/'+ 'Gossiping' +'/index.html',
    'yes':'yes' 
    }

    #破除18歲的限制
    rs = requests.session()
    req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
    req = rs.get('https://www.ptt.cc/bbs/'+ 'Gossiping' +'/index.html',verify = False)
    req = rs.get(post_url, verify=False)

    if req.status_code==200:
        web_content = req.text
        #以美味湯解析html
        soup = BeautifulSoup(web_content, 'lxml')
        main_content = soup.find('div',id="main-content")
        metas = main_content.select('div.article-metaline')

        author = metas[0].select('span.article-meta-value')[0].text
        title = metas[1].select('span.article-meta-value')[0].text
        post_date = metas[2].select('span.article-meta-value')[0].text
        author_ID = author.split('(',1)[0].replace(' ','')
        author_name = author.split('(',1)[1].replace(')','')

        #content篩選出文章內文
        content = soup.find(id="main-content").text
        target_content=u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        date = soup.select('.article-meta-value')[3].text
        content = content[0].split(date)
        content = content[1].replace('\n', '  ').replace('\t', '  ')

        #note = soup.select('span.f2')
        #org_url = note[1].text.replace('※ 文章網址: ','')
        createdTime = dt.datetime.today().strftime("%d/%m/%Y %H:%M:%S")
        post_info = []
        post_info.append({'authorId':author_ID, 'authorName':author_name, 'title':title, 'publishedTime':post_date
                          , 'content':content, 'canonicalUrl':post_url, 'createdTime':createdTime})
        post_info = pd.DataFrame(post_info)

        pushes = main_content.find_all('div', class_='push')
        messages = []
        for push in pushes:
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            # if find is None: find().strings -> list -> ' '.join; else the current way
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')    
            messages.append( {'canonicalUrl':post_url, 'push_tag': push_tag, 'commentId': push_userid
                              , 'commentContent': push_content, 'commentTime': push_ipdatetime} )

        messages = pd.DataFrame(messages)

        #post_data = pd.merge(post_info, messages, how='left', on='canonicalUrl')

        return post_info, messages

    else:
        return "404"

In [134]:
get_post_data('https://www.ptt.cc/bbs/Gossiping/M.1583233903.A.A4C.html')[1]

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,WolfTeacher,嘔嘔嘔嘔,180.176.0.145 03/03 19:11
1,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,foolfighter,衣索比亞狗,223.137.205.227 03/03 19:12
2,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,kenek0612,五樓的洨,39.12.42.254 03/03 19:12
3,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,DiMammaMia,送他上西天,114.136.47.217 03/03 19:12
4,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,kaiyao,送他巧克力，祝他要到手指,111.241.103.23 03/03 19:12
5,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,will0620,祖國的善液,223.136.24.151 03/03 19:12
6,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,→,Yude0109,R.I.P,59.115.68.236 03/03 19:12
7,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,railwaytaiwa,祝他像老柯一樣拉肚……,36.237.211.250 03/03 19:12
8,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,patiger,請他吃拔辣,114.39.84.185 03/03 19:12
9,https://www.ptt.cc/bbs/Gossiping/M.1583233903....,推,hqu,阿兩也是這一天生日,42.77.65.70 03/03 19:13


# 再寫一個可以看要爬幾頁的function，產出href_list

In [3]:
def get_href_from_page(board_name, scrap_page):
    payload={
    'from':'/bbs/'+ board_name +'/index.html',
    'yes':'yes' 
    }

    #破除18歲的限制
    rs = requests.session()
    req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
    req = rs.get('https://www.ptt.cc/bbs/'+ board_name +'/index.html',verify = False)
    if req.status_code==200:
        web_content = req.text
        #以美味湯解析html
        soup = BeautifulSoup(web_content, 'lxml')
        pre_num = soup.select('div.btn-group > a')[3]['href'].replace('/bbs/'+board_name+'/index','') ##
        pre_num = pre_num.replace('.html','')
        total_page_index = pd.to_numeric(pre_num)+1

        url_list = []
        for i in range(total_page_index, total_page_index-scrap_page, -1):
            req = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
            req = rs.get('https://www.ptt.cc/bbs/'+ board_name +'/index'+str(i)+'.html',verify = False)
            web_content = req.text
            #以美味湯解析html
            soup = BeautifulSoup(web_content, 'lxml')
            post_list = soup.find_all('div',class_="title")
            for item in post_list:
                try:
                    url_list.append('https://www.ptt.cc'+item.select_one("a").get("href"))
                except: #刪文抓不到href
                    pass         
        return url_list
    
    else:
        return "404"

# 開始寫 main

In [7]:
def main(Board_Name, Scrap_Page):
    post_list = get_href_from_page(board_name=Board_Name, scrap_page=Scrap_Page)
    all_post_info = get_post_data(post_list[0])[0] 
    for i in range(1, len(post_list)):
        try:
            all_post_info = all_post_info.append(get_post_data(post_list[i])[0])
        except:
            pass

    all_msg = get_post_data(post_list[0])[1] 
    for i in range(1, len(post_list)):
        try:
            all_msg = all_msg.append(get_post_data(post_list[i])[1])    
        except:
            pass
    all_post_info.to_csv('all_post_info.csv')
    all_msg.to_csv('all_msg.csv')
        
    return all_post_info, all_msg

In [8]:
main('AGEC88',1)[0] 

,authorId,authorName,canonicalUrl,content,createdTime,publishedTime,title
0,BABYMING,you're my treasure,https://www.ptt.cc/bbs/AGEC88/M.1326934346.A.5...,今年要不要辦班聚阿？ 感覺上次聚會又過兩年了 -- ※ 發信站: 批踢...,04/03/2020 17:40:05,Thu Jan 19 08:52:24 2012,班聚？？
0,dfj,。與主同行。,https://www.ptt.cc/bbs/AGEC88/M.1327038642.A.3...,※ 引述《BABYMING (you're my treasure)》之銘言： : 今...,04/03/2020 17:40:05,Fri Jan 20 13:50:39 2012,Re: 班聚？？
0,dfj,。與主同行。,https://www.ptt.cc/bbs/AGEC88/M.1348991955.A.E...,下午為了記起一個遺忘的名字， 看了一下班版的精華區， 發現以前的我們真的...,04/03/2020 17:40:06,Sun Sep 30 15:59:06 2012,[閒聊] 以前的我們好青春喔！
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1356447777.A.5...,"還有人在看這個版嗎??? 大家快去收信 更新一下通訊錄吧~ 目前只有""...",04/03/2020 17:40:07,Tue Dec 25 23:02:54 2012,通訊錄更新
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1357044966.A.6...,呵呵 又過一年了 -- ※ 發信站: 批踢踢實業坊(ptt.cc) ◆ F...,04/03/2020 17:40:07,Tue Jan 1 20:56:03 2013,新年快樂
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1360712747.A.C...,※ 引述《avstar (天涼好個秋)》之銘言： : 呵呵 : 又過一年了 又...,04/03/2020 17:40:08,Wed Feb 13 07:45:45 2013,Re: 新年快樂
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1376066385.A.F...,※ 引述《avstar (天涼好個秋)》之銘言： : 還有人在看這個版嗎??? : ...,04/03/2020 17:40:09,Sat Aug 10 00:39:43 2013,Re: 通訊錄更新
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1392388175.A.4...,吃元宵囉~ -- ※ 發信站: 批踢踢實業坊(ptt.cc) ◆ From:...,04/03/2020 17:40:09,Fri Feb 14 22:29:31 2014,情人節快樂
0,BABYMING,you're my treasure,https://www.ptt.cc/bbs/AGEC88/M.1393975370.A.9...,大家好 我是小明，雖然現在來這的人少了 我還是先告知 我在找時間來辦同學會 目前...,04/03/2020 17:40:10,Wed Mar 5 07:22:48 2014,[公告] 本年度同學會
0,avstar,天涼好個秋,https://www.ptt.cc/bbs/AGEC88/M.1398493686.A.5...,※ 引述《BABYMING (you're my treasure)》之銘言： : 大...,04/03/2020 17:40:11,Sat Apr 26 14:28:02 2014,Re: [公告] 本年度同學會


In [239]:
main('AGEC88',5)[1] 

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/AGEC88/M.1348991955.A.E...,推,BABYMING,我也很高興跟大家當同學^^,10/01 21:46
0,https://www.ptt.cc/bbs/AGEC88/M.1356447777.A.5...,推,dfj,好期待「十週年畢業感恩同學會」喔！,12/27 16:47
0,https://www.ptt.cc/bbs/AGEC88/M.1376066385.A.F...,推,dfj,同學會呢？,08/30 13:03
0,https://www.ptt.cc/bbs/AGEC88/M.1392388175.A.4...,推,dfj,祝大家天天快樂！,02/21 12:44
0,https://www.ptt.cc/bbs/AGEC88/M.1393975370.A.9...,推,dfj,辛苦了!,04/01 16:13
...,...,...,...,...,...
5,https://www.ptt.cc/bbs/AGEC88/M.1169655425.A.7...,→,indy,有空過年前湊一團來個聚餐吧!!,02/08 23:04
6,https://www.ptt.cc/bbs/AGEC88/M.1169655425.A.7...,推,dfj,轉眼間，好快就要過年了耶..好像也沒什麼消息..,02/11 01:11
0,https://www.ptt.cc/bbs/AGEC88/M.1171723570.A.B...,推,dfj,May God Bless All of you!!,02/26 11:21
1,https://www.ptt.cc/bbs/AGEC88/M.1171723570.A.B...,推,chiwhe921,艀w,10/10 23:32


In [6]:
main('sex',2)[0]

,authorId,authorName,canonicalUrl,content,createdTime,publishedTime,title
0,IonaIona,Iona,https://www.ptt.cc/bbs/sex/M.1583295193.A.FFF....,昨天跟一群酒肉朋友聊到這個話題 發現有些女孩兒雖然不是蕾絲邊 但會挑蕾絲邊的片片看 ...,04/03/2020 17:36:15,Wed Mar 4 12:13:11 2020,[閒聊] 男女生喜歡看的A片真的會有差別嗎?
0,ginko59,Ginko,https://www.ptt.cc/bbs/sex/M.1583297745.A.42B....,大家午安 小妹169/58... 嗯 不是重點 這個月陸續在花X bbs跟J開...,04/03/2020 17:36:16,Wed Mar 4 12:55:43 2020,Re: [閒聊] 約炮app是不是很難約到炮？
0,hvariables,Speculative Male,https://www.ptt.cc/bbs/sex/M.1583300261.A.990....,https://www.chinatimes.com/realtimenews/2020...,04/03/2020 17:36:17,Wed Mar 4 13:37:38 2020,[新聞] 色狼偷窺裙底 警3小時速逮
0,pzhao,睡覺青年,https://www.ptt.cc/bbs/sex/M.1583303946.A.557....,男女還是有差，我滑過幾個女生朋友的Tinder ，每個都99+、每個配對人數都破表，拉...,04/03/2020 17:36:17,Wed Mar 4 14:39:04 2020,Re: [閒聊] 約炮app是不是很難約到炮？
0,ytkuang,就是那個光,https://www.ptt.cc/bbs/sex/M.1583303984.A.DCE....,人妻與小王「玩屁股」 丈夫看鹹濕對話氣炸了 https://news.ltn.com....,04/03/2020 17:36:18,Wed Mar 4 14:39:41 2020,[新聞] 人妻與小王「玩屁股」 丈夫看鹹濕對話氣
0,Beanoodle,津崎似超人平匡先生,https://www.ptt.cc/bbs/sex/M.1583306018.A.1C6....,※ 引述《WuDangJie (國安雞精準備進場)》之銘言： : 大家晚安 : 小弟...,04/03/2020 17:36:18,Wed Mar 4 15:13:36 2020,Re: [閒聊] 約炮app是不是很難約到炮？
0,Beanoodle,津崎似超人平匡先生,https://www.ptt.cc/bbs/sex/M.1583312168.A.B47....,大家安安 乳提 以前什麼都不懂的時候 一定是買杜雷斯 後來發...,04/03/2020 17:36:19,Wed Mar 4 16:56:06 2020,[閒聊] 保險套會注重啥?
0,lpbrother,LP哥(LP = Love & Peace,https://www.ptt.cc/bbs/sex/M.1583314231.A.470....,有差啊， 鄉民最愛看小護士系列 還有空姐系列， 而且很奇怪的是A片...,04/03/2020 17:36:20,Wed Mar 4 17:30:29 2020,Re: [閒聊] 男女生喜歡看的A片真的會有差別嗎?
0,FallRed,☆╮落紅a圓仔╭㊣,https://www.ptt.cc/bbs/sex/M.1564409000.A.CD1....,※ [本文轉錄自 SYSOP 看板 #1TD-O6jj ] 作者: longbow...,04/03/2020 17:36:21,Mon Jul 29 22:03:19 2019,Fw: [公告] 請使用者多加注意我國保護兒少的法令
0,Assisi,Francesco d'Assisi,https://www.ptt.cc/bbs/sex/M.1579516428.A.917....,本著欠公告不要欠過年的精神 (?) 以下是本次冬季徵文的結果 ☺ ☺...,04/03/2020 17:36:22,Mon Jan 20 18:33:45 2020,[公告] 2019 冬季徵文 結果 暨 得獎名單


In [227]:
get_post_data('https://www.ptt.cc/bbs/sex/M.1583221229.A.059.html')[0] 

,authorId,authorName,title,publishedTime,content,canonicalUrl,createdTime
0,goldenrose,蓋世英雄,Re: [討論] 女生願意被幹嘴巴的比例高嗎,Tue Mar 3 15:40:27 2020,因為本身是口交控(含到底超讚) 看到這篇實在忍不住想回 然後我是男生 不要看第一句...,https://www.ptt.cc/bbs/sex/M.1583221229.A.059....,03/03/2020 21:47:25


In [139]:
AGEC_list = get_href_from_page(board_name='AGECservice', scrap_page=1)

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [158]:
AGEC_list[0]

'https://www.ptt.cc/bbs/AGECservice/M.1369038539.A.72F.html'

In [174]:
all_post_info = get_post_data(AGEC_list[0])[0] 
for i in range(1, len(AGEC_list)):
    all_post_info = all_post_info.append(get_post_data(AGEC_list[i])[0])



/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Mac/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847

In [185]:
all_msg = get_post_data(AGEC_list[0])[1] 
for i in range(1, len(AGEC_list)):
    all_msg = all_msg.append(get_post_data(AGEC_list[i])[1])

In [186]:
all_msg

,canonicalUrl,push_tag,commentId,commentContent,commentTime
0,https://www.ptt.cc/bbs/AGECservice/M.137137434...,推,yf87008,cool,06/18 01:26
0,https://www.ptt.cc/bbs/AGECservice/M.137205577...,推,etwu0718,,06/24 20:29
0,https://www.ptt.cc/bbs/AGECservice/M.133104532...,推,achun8184,夾夾讚 頭推,03/06 23:57
1,https://www.ptt.cc/bbs/AGECservice/M.133104532...,→,onemoreshot,快點開工開始做事比較實在！有問題快問,03/07 01:11
2,https://www.ptt.cc/bbs/AGECservice/M.133104532...,推,pgking1224,何世堯大教案好像是管理組...,03/15 22:07
0,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,bloodyjimi,3/26 農藝組 10:00~12:00 13:00~17:00 tks!,03/16 14:58
1,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,hauerhsieh,3/22 全天共六小時 B97607019 農經四 謝政豪 謝夾,03/20 14:10
2,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,4/4 農藝組 1000-1200 1300-1500 農經二 黃崇榮 謝謝！,03/25 20:10
3,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,不好意思 想提前一天 改成4/3(二) 謝謝！,03/28 07:23
4,https://www.ptt.cc/bbs/AGECservice/M.133183207...,推,syric,上次登記春假但農場沒開 想再登記4/20(五) 1300-1700 謝謝!,04/13 13:21


In [2]:
pd.read_csv('all_post_info.csv')

,Unnamed: 0,authorId,authorName,canonicalUrl,content,createdTime,publishedTime,title
0,0,angela0615,菜菜,https://www.ptt.cc/bbs/AGEC95/M.1285307144.A.4...,系辦這裡有你的掛號信喔!! 有空來系辦領信吧!! =] -- ※ ...,05/03/2020 07:44:30,Fri Sep 24 13:45:41 2010,林鎂茹學姐的信 =]
1,0,ilikecc,cchuang,https://www.ptt.cc/bbs/AGEC95/M.1285327111.A.F...,受人委託徵才~ 資拓股份有限公司，是一家與台灣電信業規模最大的中華電信股份有限...,05/03/2020 07:44:31,Fri Sep 24 19:18:29 2010,[徵才]農業資訊管理人才
2,0,t0ts5,好人難作,https://www.ptt.cc/bbs/AGEC95/M.1287992567.A.A...,安安喔大家 我是新來的 -- ※ 發信站: 批踢踢實業坊(ptt.cc) ...,05/03/2020 07:44:32,Mon Oct 25 15:42:44 2010,[問題] 問題
3,0,nowitzki41x,小球,https://www.ptt.cc/bbs/AGEC95/M.1288601712.A.F...,請 康維娟 同學 黃紀穎 同學 陳培甄 同學 至...,05/03/2020 07:44:32,Mon Nov 1 16:55:10 2010,[情報] 書卷獎獎學金領取
4,0,david00129,二家展爺發揮創s意，與武,https://www.ptt.cc/bbs/AGEC95/M.1303844697.A.8...,中華青年企業家協會 穿梭百年-中山先生革命足跡之旅 ＊ ＊ 一路向北，兩岸四地的...,05/03/2020 07:44:33,Wed Apr 27 03:04:54 2011,[好康]暑假交流團-穿梭百年-中山先生足跡之旅
5,0,bofgb,..,https://www.ptt.cc/bbs/AGEC95/M.1304954859.A.A...,▅▃▃▅ ▅▃▃▅ ▅▃▃▅ ▅...,05/03/2020 07:44:34,Mon May 9 23:27:38 2011,AIESEC許自己一個世界級的成年禮-暑期海外成長計畫
6,0,jkwhich,which,https://www.ptt.cc/bbs/AGEC95/M.1305644873.A.B...,※ [本文轉錄自 female_club 看板 #1DoxYpEK ] 作者: j...,05/03/2020 07:44:37,Tue May 17 23:07:52 2011,[轉錄][網宣] 伴侶盟校園巡迴《扣押幸福》映後座談會
7,0,Delightme,恬淡生活,https://www.ptt.cc/bbs/AGEC95/M.1308457055.A.9...,這裡怎麼人那麼少呢?? 加油~~加油啊!!! 希望大家偶爾來走走逛逛、灌灌水、衝...,05/03/2020 07:44:40,Sun Jun 19 12:17:32 2011,[閒聊] 這裡的人氣愈來愈少囉
8,0,PTTjane,jane wu,https://www.ptt.cc/bbs/AGEC95/M.1412821405.A.C...,各位農經系的學長姐大家好~~ 小妹是103年系學會新任資訊部部長 今年我們的工作重點...,05/03/2020 07:44:41,Thu Oct 9 10:23:22 2014,[情報] 農經粉專上線囉！
9,0,yuiismywife,新垣結衣我老婆,https://www.ptt.cc/bbs/AGEC95/M.1501380535.A.1...,不好意思，想請問農經系 的大家，我有個朋友想準備 104年的農經所在職專...,05/03/2020 07:44:42,Sun Jul 30 10:08:52 2017,[請益] （代ＰＯ） 農經所在職專班準備


In [32]:
random.uniform(0,2)

2.4756669125066937